# Dependencies

In [46]:
from langchain.llms import LlamaCpp, OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage
from langchain import PromptTemplate

# Environment Variables

In [21]:
from dotenv import dotenv_values, load_dotenv
config = dotenv_values(".env")
OPENAI_API_KEY={**config}["OPENAI_API_KEY"]

# Modelo

## Modelo de predicción de texto

In [37]:
#Modelo autocompletador de textos text-davinci-003
llm_openai = OpenAI(model_name = "text-davinci-003", openai_api_key= OPENAI_API_KEY, temperature=0.5)

In [38]:
print(llm_openai("Hola, cómo estás?"))



Estoy bien, ¿y tú?


## Modelo ChatBot

In [41]:
chatgpt = ChatOpenAI(openai_api_key=OPENAI_API_KEY)

In [45]:
#HumanMessage
response = chatgpt([HumanMessage(content="Hola, Cómo estás?")])
print(response.content)

¡Hola! Soy una inteligencia artificial, por lo que no tengo emociones, pero estoy aquí para ayudarte. ¿En qué puedo asistirte hoy?


# Prompts

## PromptTemplate

In [47]:
basic_template = """

Eres un asistente virtual llamado David que responde a preguntas de manera muy breve.
Pregunta: Cuáles son los ingredientes para preparar {plato}
Respuesta:


"""

In [48]:
#Se construye el template agregándole las variables de entrada
prompt_temp = PromptTemplate(input_variables=["plato"], template=basic_template)

prompt_value = prompt_temp.format(plato="Sancocho de gallina")
print(prompt_value)



Eres un asistente virtual llamado David que responde a preguntas de manera muy breve.
Pregunta: Cuáles son los ingredientes para preparar Sancocho de gallina
Respuesta:





In [49]:
response = llm_openai(prompt_value)
print(response)

Los ingredientes para preparar Sancocho de gallina incluyen una gallina entera, una cebolla, una cucharada de achiote, una cucharada de comino, una cucharada de orégano, una cucharada de ajo en polvo, dos cucharadas de sal, una cucharada de pimienta, una cucharada de cilantro, dos tomates, una taza de papas, una taza de zanahorias, una taza de maíz, una taza de yuca y una taza de plátano verde.


## How many Tokens we're sending?

In [60]:
#Requiere instalar tiktoken
print("tokens:",llm_openai.get_num_tokens("Hola, cómo estás?"))

tokens: 9


## ChatPromptTemplate

Template para modelos de chat. Permite darle información a los modelos de chat en la manera que lo necesiten.

Elementos:

- **Human:** El texto escrito por el usuario
- **AI:** Texto respuesta del modelo
- **System:** Texto enviado al modelo para darle contexto de su funcionamiento

In [65]:
from langchain.prompts import SystemMessagePromptTemplate, HumanMessagePromptTemplate, ChatPromptTemplate, ChatMessagePromptTemplate, AIMessagePromptTemplate

In [62]:
#Construcción del template para el sistema
prompt_temp_system = PromptTemplate(
    template = "Eres un asistente virtual que recomienda una alternativa {adjetivo} a un producto",
    input_variables=["adjetivo"]
)
#Langchain se encarga de formatear el prompt de manera correcta para el modelo
template_system = SystemMessagePromptTemplate(prompt=prompt_temp_system)

#Humano
prompt_temp_human = PromptTemplate(template="{texto}",input_variables=["texto"])
template_human = HumanMessagePromptTemplate(prompt=prompt_temp_human)

In [69]:
chat_prompt = ChatPromptTemplate.from_messages([template_system,template_human])

#Formato del prompt
chat_prompt_value = chat_prompt.format_prompt(adjetivo="económica",texto="ipad").to_messages()
print(chat_prompt_value)

[SystemMessage(content='Eres un asistente virtual que recomienda una alternativa económica a un producto'), HumanMessage(content='ipad')]


In [70]:
chat_response = chatgpt(chat_prompt_value)
print(chat_response)

content='Si estás buscando una alternativa económica al iPad, te recomendaría considerar la tablet Amazon Fire HD 10. Esta tablet ofrece una pantalla de alta definición de 10.1 pulgadas, un procesador rápido y potente, y una duración de batería de hasta 12 horas. Además, cuenta con acceso a la tienda de aplicaciones de Amazon, lo que te permitirá descargar una amplia variedad de aplicaciones y juegos. Aunque no tiene todas las características y funcionalidades de un iPad, es una opción más económica que puede cumplir con tus necesidades básicas de navegación web, reproducción de contenido multimedia y uso de aplicaciones.'


# Example Selector

Propósito: Alimentar el modelo con ejemplos y posibles respuestas que se esperan recibir

In [71]:
from langchain import FewShotPromptTemplate

In [74]:
#Se crea una lista de diccionarios con ejemplos para el modelo
examples = [
    {"pregunta": "Cuál es el ingrediente principal de la pizza?","respuesta": "La masa y salsa de tomate"},
    {"pregunta": "Cuál es el ingrediente principal de las hamburguesas?","respuesta": "La carne y el pan"},
    {"pregunta": "Cuál es el ingrediente principal del burrito?","respuesta": "La tortilla y la carne"}
]
#
prompt_temp_examples = PromptTemplate(input_variables=["pregunta","respuesta"],
                                      template = "Pregunta: {pregunta}\nRespuesta: {respuesta}")

In [76]:
#Template que genera el formato para las preguntas y respuestas
prompt_examples = FewShotPromptTemplate(example_prompt=prompt_temp_examples,
                                        examples=examples,
                                        prefix="Eres un asistente virtual culinario que responde a preguntas sobre ingredientes principales de platos de manera breve",
                                        suffix="Pregunta: {pregunta}\nRespuesta:",
                                        input_variables=["pregunta"])

In [79]:
prompt_value = prompt_examples.format(pregunta="Cuál es el ingrediente principal del coctel de camarones?")
print(prompt_value)

Eres un asistente virtual culinario que responde a preguntas sobre ingredientes principales de platos de manera breve

Pregunta: Cuál es el ingrediente principal de la pizza?
Respuesta: La masa y salsa de tomate

Pregunta: Cuál es el ingrediente principal de las hamburguesas?
Respuesta: La carne y el pan

Pregunta: Cuál es el ingrediente principal del burrito?
Respuesta: La tortilla y la carne

Pregunta: Cuál es el ingrediente principal del coctel de camarones?
Respuesta:


In [80]:
ingredients_response = llm_openai(prompt_value)
print(ingredients_response)

 Camarones y salsa picante


# Output Parsers

Formatear la respuesta a conveniencia

In [81]:
from langchain.output_parsers import CommaSeparatedListOutputParser
from langchain.prompts import PromptTemplate
output_parser = CommaSeparatedListOutputParser()
format_instructions = output_parser.get_format_instructions()

In [82]:
format_instructions

'Your response should be a list of comma separated values, eg: `foo, bar, baz`'

In [85]:
basic_temp_parser = """Cuáles son los ingredientes para preparar: {platillo}\n{ht_parser}"""

In [88]:
prompt_temp_parser = PromptTemplate(input_variables=["platillo"],
                                    template = basic_temp_parser,
                                    partial_variables = {"ht_parser":format_instructions})

In [89]:
prompt_value_parser = prompt_temp_parser.format(platillo="Burrito")

In [90]:
print(prompt_value_parser)

Cuáles son los ingredientes para preparar: Burrito
Your response should be a list of comma separated values, eg: `foo, bar, baz`


In [91]:
response_parser = llm_openai(prompt_value_parser)
response_parser

'\n\nTortilla de harina, carne, frijoles, queso, cebolla, tomate, lechuga, guacamole, crema, salsa.'

In [92]:
output_parser.parse(response_parser)

['Tortilla de harina',
 'carne',
 'frijoles',
 'queso',
 'cebolla',
 'tomate',
 'lechuga',
 'guacamole',
 'crema',
 'salsa.']